deep research就是 planning + execute + synthesize

In [18]:
#model init
import os
from dotenv import load_dotenv
load_dotenv()

from langchain.chat_models import init_chat_model

qwen_plus_model = init_chat_model(
    "qwen-plus",
    model_provider="openai",
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
    api_key=os.getenv("DASHSCOPE_API_KEY"),
    temperature=0.5
)


In [19]:
#response_format

from pydantic import  BaseModel,Field

class WebSearchItem(BaseModel):
    """网页搜索项"""
    query:str = Field(...,description = "用于网页搜索的关键词，只写一个关键词")
    reason: str = Field(...,description="该搜索词对回答原问题的必要说明")

class WebSearchPlan(BaseModel):
    """网页搜索计划（包含多条搜索指令）"""
    searchs: list[WebSearchItem] = Field(...,description="为精确回答原问题需执行的网页搜索列表")

In [20]:
from langchain.agents import create_agent

system_prompt = """
    你是一个专业的研究助手，需基于用户查询完成以下核心任务，确保搜索的精准高效：
    1.设计10个不重复的网页搜索关键词，关键词需贴合查询核心需求，兼顾精准度与覆盖性；
    2.为每个关键词单独说明搜索原因，说明该关键词如何助理解答用户查询，原因需具体可落地，不笼统。
"""

plainning_agent = create_agent(
    model = qwen_plus_model,
    system_prompt = system_prompt,
    response_format = WebSearchPlan
)   

In [ ]:
response = plainning_agent.invoke(
    {
        "messages":[
            {"role":"user","content":"AI在教育中的应用"}
        ]
    }
)
search_plan = response["structured_response"]
queries = [item for item in search_plan.searchs]
queries

SyntaxError: invalid syntax (813789296.py, line 8)